In [7]:
import torchvision
import torchvision.transforms as transforms
# Data Augmentations 
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [8]:
# Creates a Quantum Neural Network (QNN) with 2 qubits
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

def create_qnn():
# # 2-qubit feature map for encoding 2 classical inputs
    feature_map = ZZFeatureMap(2)

# RealAmplitudes adds parameterized rotation gates that will be trained
    ansatz = RealAmplitudes(2, reps=1)

    qc = QuantumCircuit(2) # Create a quantum circuit
    qc.compose(feature_map, inplace=True) # First encode the data
    qc.compose(ansatz, inplace=True) # Then apply trainable gates

    # Create estimator
    estimator = Estimator()

    # Create the Quantum Neural Network
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
        estimator=estimator,
    )
    return qnn

In [9]:
import torch.nn as nn
import torch.nn.functional as F
from torch import cat

class QuantumHybridCNN(nn.Module):
    def __init__(self, qnn):
        super().__init__()
        # 1st Convo layer
        self.conv1 = nn.Conv2d(1, 2, kernel_size=5)
        # Input: Takes one Greyscale image as Input
        # Output:  2 feature maps
        # Kernel: 5x5

        # 2nd Convo layer
        self.conv2 = nn.Conv2d(2, 16, kernel_size=5) #2nd Convo layer
        # Input: 2 input channel
        # Output:  16 feature maps
        # Kernel: 5x5

        # regularization : Drupout Layer
        self.dropout = nn.Dropout2d()
        
        # 1st Fully connected layer:
        # Flat Convolutional features (400) ---> 64 features
        # converts convolutional features to dense representation
        # For 32x32 CIFAR images, after convolutions and pooling we get 16x5x5=400 features
        self.fc1 = nn.Linear(400, 64)

        # 2nd Fully connected layer:
        # 64 features ----> 2 features
        self.fc2 = nn.Linear(64, 2)

        # Quantum part
        self.qnn = TorchConnector(qnn) #TorchConnector wraps the quantum circuit
        

        # 4th Fully Connected Layer:
        # 1 feature ------> 2 features
        self.fc3 = nn.Linear(1, 1)

    def forward(self, x):

        # 1st Layer
        x = F.relu(self.conv1(x))# Relu Function
        x = F.max_pool2d(x, 2)# Max Pooling

        # 2nd Convv Layer 
        x = F.relu(self.conv2(x)) #ReLU activation
        x = F.max_pool2d(x, 2) #Max Pooling

        # dropout for regularization
        x = self.dropout(x)
        
        # Flatten layer
        # Flatten the 3D tensor to 1D for dense layers
        x = x.view(x.shape[0], -1) # Shape becomes [batch_size, 400]

        # 1st Fully connected layer:
        x = F.relu(self.fc1(x))# ReLU

        # 2nd Fully connected layer:
        # prepares data for quantum circuit
        x = self.fc2(x) # No activation

        # 3rd Layer 
        # Quantum processing
        # data passes through the quantum neural network
        x = self.qnn(x)

        # 4th Fully Connected Layer:
        x = self.fc3(x)
        
        # Unlike the classical model's softmax, this creates [p, 1-p] directly
        return cat((x, 1 - x), -1)

In [10]:
from torchinfo import summary
demo_qnn = create_qnn()
demo_Model = QuantumHybridCNN(demo_qnn)
summary(demo_Model, input_size=(1, 1, 32, 32))  # CIFAR-10 images: 32×32

/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_2139/257468979.py:20: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_2139/257468979.py:23: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Layer (type:depth-idx)                   Output Shape              Param #
QuantumHybridCNN                         [1, 2]                    --
├─Conv2d: 1-1                            [1, 2, 28, 28]            52
├─Conv2d: 1-2                            [1, 16, 10, 10]           816
├─Dropout2d: 1-3                         [1, 16, 5, 5]             --
├─Linear: 1-4                            [1, 64]                   25,664
├─Linear: 1-5                            [1, 2]                    130
├─TorchConnector: 1-6                    [1, 1]                    4
├─Linear: 1-7                            [1, 1]                    2
Total params: 26,668
Trainable params: 26,668
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.15
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.11
Estimated Total Size (MB): 0.14

In [11]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


In [12]:
def generate_combinations(digits=10, length=2):
    """
    Generate combinations of CIFAR-10
    """
    result = []    
    if length == 2:
        for i in range(digits):
            for j in range(i + 1, digits):
                result.append([i, j])
    else:
        def backtrack(current_combination, start_index):
            if len(current_combination) == length:
                result.append(current_combination[:])
                return
            for i in range(start_index, digits):
                current_combination.append(i)
                backtrack(current_combination, i + 1)
                current_combination.pop()
        
        backtrack([], 0)
    
    return result
a = generate_combinations()
print(f"Number of combinations: {len(a)}")

Number of combinations: 45


In [13]:
import torch
from sklearn.metrics import classification_report

def evaluate(class_indices):
    print(f"\n\n=== Evaluating classes {class_indices} ===")    
# All Possible classes       
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# Select Class     
    selected_classes = [class_names[idx] for idx in class_indices]
    print(f"Selected classes: {selected_classes}")

# Training    
# Filter data    
    filtered_train_data = [(image, label) for image, label in trainset if label in class_indices]
    filtered_train_images, filtered_train_labels = zip(*filtered_train_data)
# Map labels to binary 
    filtered_train_labels = [0 if label == class_indices[0] else 1 for label in filtered_train_labels]

# Testing 
# Filter data
    filtered_test_data = [(image, label) for image, label in testset if label in class_indices]
    filtered_test_images, filtered_test_labels = zip(*filtered_test_data)

# Map labels to binary 
    filtered_test_labels = [0 if label == class_indices[0] else 1 for label in filtered_test_labels]

# Create datasets and loaders
    train_dataset = torch.utils.data.TensorDataset(
        torch.stack(filtered_train_images),
        torch.tensor(filtered_train_labels)
        )

    test_dataset = torch.utils.data.TensorDataset(
        torch.stack(filtered_test_images),
        torch.tensor(filtered_test_labels)
    )

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    

# Use the best available device to run 
# MPS for Mac, CUDA for NVIDIA GPU, or CPU    
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create quantum circuit and model
    qnn = create_qnn()
    model = QuantumHybridCNN(qnn).to(device)

    criterion = nn.NLLLoss() # loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer
    num_epochs = 10 # Number of Epochs

    # Training of Model 
    model.train()
    for epoch in range(num_epochs):
        total_loss = []
        for batch_idx, (data, target) in enumerate(train_loader):
            # Move data to device
            data, target = data.to(device), target.to(device)
            # Zero gradients before backward pass
            optimizer.zero_grad(set_to_none=True)

            output = model(data) # Forward pass
            loss = criterion(output, target) # Calculate loss
            loss.backward() # Backward pass  
            optimizer.step() # Update weights
            total_loss.append(loss.item()) # Track loss
        
            # Track Progress:
            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_loader)}] '
                  f'Loss: {loss.item():.4f}')
    
        avg_loss = sum(total_loss) / len(total_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

    print('Finished Training')
    
    def evaluate_model(model, test_loader, device): # Evaluate Model
        model.eval()
        true_labels = []
        pred_labels = []
    
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device) # Get highest probability class
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                true_labels.extend(labels.cpu().numpy())
                pred_labels.extend(predicted.cpu().numpy())
    
        
        return classification_report(true_labels, pred_labels, target_names=selected_classes)
    
        
        
    # Evaluate and print model performance on test set    
    print("\nTest Set Performance:")
    a = evaluate_model(model, test_loader, device)
    print(a)

    # Evaluate and print model performance on training set
    print("\nTraining Set Performance:")
    b = evaluate_model(model, train_loader, device)
    print(b)
    
    # Return both performance reports
    return (a, b)

In [14]:
evaluate([0,1])



=== Evaluating classes [0, 1] ===
Selected classes: ['airplane', 'automobile']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_2139/257468979.py:20: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_2139/257468979.py:23: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6295
Epoch [1/10] Batch [200/313] Loss: -0.8220
Epoch [1/10] Batch [300/313] Loss: -0.9670
Epoch [1/10], Average Loss: -0.8089
Epoch [2/10] Batch [100/313] Loss: -1.0487
Epoch [2/10] Batch [200/313] Loss: -1.1897
Epoch [2/10] Batch [300/313] Loss: -1.3859
Epoch [2/10], Average Loss: -1.1303
Epoch [3/10] Batch [100/313] Loss: -1.2081
Epoch [3/10] Batch [200/313] Loss: -1.3448
Epoch [3/10] Batch [300/313] Loss: -1.0983
Epoch [3/10], Average Loss: -1.4172
Epoch [4/10] Batch [100/313] Loss: -1.6744
Epoch [4/10] Batch [200/313] Loss: -1.4811
Epoch [4/10] Batch [300/313] Loss: -2.0518
Epoch [4/10], Average Loss: -1.6502
Epoch [5/10] Batch [100/313] Loss: -1.9947
Epoch [5/10] Batch [200/313] Loss: -2.0069
Epoch [5/10] Batch [300/313] Loss: -2.0027
Epoch [5/10], Average Loss: -1.8793
Epoch [6/10] Batch [100/313] Loss: -1.9751
Epoch [6/10] Batch [200/313] Loss: -1.7674
Epoch [6/10] Batch [300/313] Loss: -2.5889
Epoch [6/10], Average Loss: -2.0929
Epoch [7/1

('              precision    recall  f1-score   support\n\n    airplane       0.83      0.91      0.87      1000\n  automobile       0.90      0.81      0.85      1000\n\n    accuracy                           0.86      2000\n   macro avg       0.86      0.86      0.86      2000\nweighted avg       0.86      0.86      0.86      2000\n',
 '              precision    recall  f1-score   support\n\n    airplane       0.83      0.90      0.86      5000\n  automobile       0.89      0.81      0.85      5000\n\n    accuracy                           0.86     10000\n   macro avg       0.86      0.86      0.86     10000\nweighted avg       0.86      0.86      0.86     10000\n')

In [ ]:
# evaluate([0,2])

In [ ]:
# evaluate([0,3])

In [ ]:
# evaluate([0,4])

In [ ]:
# evaluate([0,5])

In [ ]:
# evaluate([0,6])

In [ ]:
# evaluate([0,7])

In [ ]:
# evaluate([0,8])

In [ ]:
# evaluate([0,9])

In [ ]:
# evaluate([1,2])

In [ ]:
# evaluate([1,3])

In [ ]:
# evaluate([1,4])

In [ ]:
# evaluate([1,5])

In [ ]:
# evaluate([1,6])

In [ ]:
# evaluate([1,7])

In [ ]:
# evaluate([1,8])

In [ ]:
# evaluate([1,9])

In [ ]:
# evaluate([2,3])

In [ ]:
# evaluate([2,4])

In [10]:
evaluate([2,5])



=== Evaluating classes [2, 5] ===
Selected classes: ['bird', 'dog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6829
Epoch [1/10] Batch [200/313] Loss: -0.6904
Epoch [1/10] Batch [300/313] Loss: -0.6552
Epoch [1/10], Average Loss: -0.5591
Epoch [2/10] Batch [100/313] Loss: -0.5102
Epoch [2/10] Batch [200/313] Loss: -0.5539
Epoch [2/10] Batch [300/313] Loss: -0.7778
Epoch [2/10], Average Loss: -0.6087
Epoch [3/10] Batch [100/313] Loss: -0.8622
Epoch [3/10] Batch [200/313] Loss: -1.0274
Epoch [3/10] Batch [300/313] Loss: -0.7208
Epoch [3/10], Average Loss: -0.7491
Epoch [4/10] Batch [100/313] Loss: -0.8033
Epoch [4/10] Batch [200/313] Loss: -1.0775
Epoch [4/10] Batch [300/313] Loss: -1.0340
Epoch [4/10], Average Loss: -0.8914
Epoch [5/10] Batch [100/313] Loss: -0.7616
Epoch [5/10] Batch [200/313] Loss: -1.3723
Epoch [5/10] Batch [300/313] Loss: -1.2461
Epoch [5/10], Average Loss: -1.0576
Epoch [6/10] Batch [100/313] Loss: -1.2514
Epoch [6/10] Batch [200/313] Loss: -1.1243
Epoch [6/10] Batch [300/313] Loss: -1.2714
Epoch [6/10], Average Loss: -1.2303
Epoch [7/1

('              precision    recall  f1-score   support\n\n        bird       0.72      0.61      0.66      1000\n         dog       0.66      0.76      0.71      1000\n\n    accuracy                           0.68      2000\n   macro avg       0.69      0.68      0.68      2000\nweighted avg       0.69      0.68      0.68      2000\n',
 '              precision    recall  f1-score   support\n\n        bird       0.73      0.64      0.68      5000\n         dog       0.68      0.76      0.72      5000\n\n    accuracy                           0.70     10000\n   macro avg       0.70      0.70      0.70     10000\nweighted avg       0.70      0.70      0.70     10000\n')

In [11]:
evaluate([2,6])



=== Evaluating classes [2, 6] ===
Selected classes: ['bird', 'frog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.5037
Epoch [1/10] Batch [200/313] Loss: -0.4766
Epoch [1/10] Batch [300/313] Loss: -0.5864
Epoch [1/10], Average Loss: -0.5067
Epoch [2/10] Batch [100/313] Loss: -0.4543
Epoch [2/10] Batch [200/313] Loss: -0.5719
Epoch [2/10] Batch [300/313] Loss: -0.4497
Epoch [2/10], Average Loss: -0.5343
Epoch [3/10] Batch [100/313] Loss: -0.5787
Epoch [3/10] Batch [200/313] Loss: -0.7205
Epoch [3/10] Batch [300/313] Loss: -0.7730
Epoch [3/10], Average Loss: -0.5986
Epoch [4/10] Batch [100/313] Loss: -0.8682
Epoch [4/10] Batch [200/313] Loss: -0.8656
Epoch [4/10] Batch [300/313] Loss: -0.7991
Epoch [4/10], Average Loss: -0.8123
Epoch [5/10] Batch [100/313] Loss: -0.9530
Epoch [5/10] Batch [200/313] Loss: -0.9712
Epoch [5/10] Batch [300/313] Loss: -1.3810
Epoch [5/10], Average Loss: -1.0306
Epoch [6/10] Batch [100/313] Loss: -1.0386
Epoch [6/10] Batch [200/313] Loss: -1.7776
Epoch [6/10] Batch [300/313] Loss: -1.7983
Epoch [6/10], Average Loss: -1.2616
Epoch [7/1

('              precision    recall  f1-score   support\n\n        bird       0.50      0.83      0.62      1000\n        frog       0.48      0.15      0.23      1000\n\n    accuracy                           0.49      2000\n   macro avg       0.49      0.49      0.43      2000\nweighted avg       0.49      0.49      0.43      2000\n',
 '              precision    recall  f1-score   support\n\n        bird       0.50      0.84      0.63      5000\n        frog       0.49      0.16      0.24      5000\n\n    accuracy                           0.50     10000\n   macro avg       0.50      0.50      0.43     10000\nweighted avg       0.50      0.50      0.43     10000\n')

In [12]:
evaluate([2,7])



=== Evaluating classes [2, 7] ===
Selected classes: ['bird', 'horse']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.7420
Epoch [1/10] Batch [200/313] Loss: -0.6681
Epoch [1/10] Batch [300/313] Loss: -0.8720
Epoch [1/10], Average Loss: -0.7238
Epoch [2/10] Batch [100/313] Loss: -1.2237
Epoch [2/10] Batch [200/313] Loss: -1.0652
Epoch [2/10] Batch [300/313] Loss: -1.2999
Epoch [2/10], Average Loss: -1.0314
Epoch [3/10] Batch [100/313] Loss: -1.0637
Epoch [3/10] Batch [200/313] Loss: -1.2857
Epoch [3/10] Batch [300/313] Loss: -1.3416
Epoch [3/10], Average Loss: -1.2604
Epoch [4/10] Batch [100/313] Loss: -1.6804
Epoch [4/10] Batch [200/313] Loss: -1.5069
Epoch [4/10] Batch [300/313] Loss: -1.9733
Epoch [4/10], Average Loss: -1.4682
Epoch [5/10] Batch [100/313] Loss: -1.0633
Epoch [5/10] Batch [200/313] Loss: -2.0706
Epoch [5/10] Batch [300/313] Loss: -2.0519
Epoch [5/10], Average Loss: -1.6650
Epoch [6/10] Batch [100/313] Loss: -2.1173
Epoch [6/10] Batch [200/313] Loss: -1.9360
Epoch [6/10] Batch [300/313] Loss: -1.0826
Epoch [6/10], Average Loss: -1.8282
Epoch [7/1

('              precision    recall  f1-score   support\n\n        bird       0.79      0.74      0.77      1000\n       horse       0.76      0.80      0.78      1000\n\n    accuracy                           0.77      2000\n   macro avg       0.77      0.77      0.77      2000\nweighted avg       0.77      0.77      0.77      2000\n',
 '              precision    recall  f1-score   support\n\n        bird       0.82      0.79      0.80      5000\n       horse       0.80      0.82      0.81      5000\n\n    accuracy                           0.81     10000\n   macro avg       0.81      0.81      0.81     10000\nweighted avg       0.81      0.81      0.81     10000\n')

In [13]:
evaluate([2,8])



=== Evaluating classes [2, 8] ===
Selected classes: ['bird', 'ship']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.9250
Epoch [1/10] Batch [200/313] Loss: -0.7406
Epoch [1/10] Batch [300/313] Loss: -0.7634
Epoch [1/10], Average Loss: -0.7792
Epoch [2/10] Batch [100/313] Loss: -1.1072
Epoch [2/10] Batch [200/313] Loss: -1.1834
Epoch [2/10] Batch [300/313] Loss: -0.9205
Epoch [2/10], Average Loss: -1.0903
Epoch [3/10] Batch [100/313] Loss: -1.4555
Epoch [3/10] Batch [200/313] Loss: -0.7089
Epoch [3/10] Batch [300/313] Loss: -1.3328
Epoch [3/10], Average Loss: -1.3239
Epoch [4/10] Batch [100/313] Loss: -1.5852
Epoch [4/10] Batch [200/313] Loss: -1.3966
Epoch [4/10] Batch [300/313] Loss: -1.3220
Epoch [4/10], Average Loss: -1.5432
Epoch [5/10] Batch [100/313] Loss: -2.2652
Epoch [5/10] Batch [200/313] Loss: -2.6058
Epoch [5/10] Batch [300/313] Loss: -2.2315
Epoch [5/10], Average Loss: -1.7699
Epoch [6/10] Batch [100/313] Loss: -2.3045
Epoch [6/10] Batch [200/313] Loss: -2.0120
Epoch [6/10] Batch [300/313] Loss: -2.3394
Epoch [6/10], Average Loss: -2.0002
Epoch [7/1

('              precision    recall  f1-score   support\n\n        bird       0.87      0.82      0.84      1000\n        ship       0.83      0.87      0.85      1000\n\n    accuracy                           0.84      2000\n   macro avg       0.85      0.84      0.84      2000\nweighted avg       0.85      0.84      0.84      2000\n',
 '              precision    recall  f1-score   support\n\n        bird       0.87      0.84      0.86      5000\n        ship       0.85      0.88      0.86      5000\n\n    accuracy                           0.86     10000\n   macro avg       0.86      0.86      0.86     10000\nweighted avg       0.86      0.86      0.86     10000\n')

In [14]:
evaluate([2,9])



=== Evaluating classes [2, 9] ===
Selected classes: ['bird', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6303
Epoch [1/10] Batch [200/313] Loss: -1.0008
Epoch [1/10] Batch [300/313] Loss: -0.9683
Epoch [1/10], Average Loss: -0.8735
Epoch [2/10] Batch [100/313] Loss: -1.2733
Epoch [2/10] Batch [200/313] Loss: -1.4613
Epoch [2/10] Batch [300/313] Loss: -1.7704
Epoch [2/10], Average Loss: -1.2837
Epoch [3/10] Batch [100/313] Loss: -1.6569
Epoch [3/10] Batch [200/313] Loss: -1.7484
Epoch [3/10] Batch [300/313] Loss: -1.5208
Epoch [3/10], Average Loss: -1.6051
Epoch [4/10] Batch [100/313] Loss: -1.5348
Epoch [4/10] Batch [200/313] Loss: -1.7978
Epoch [4/10] Batch [300/313] Loss: -1.8638
Epoch [4/10], Average Loss: -1.8860
Epoch [5/10] Batch [100/313] Loss: -1.6880
Epoch [5/10] Batch [200/313] Loss: -2.4188
Epoch [5/10] Batch [300/313] Loss: -2.9375
Epoch [5/10], Average Loss: -2.1592
Epoch [6/10] Batch [100/313] Loss: -2.7075
Epoch [6/10] Batch [200/313] Loss: -2.0844
Epoch [6/10] Batch [300/313] Loss: -2.1089
Epoch [6/10], Average Loss: -2.4158
Epoch [7/1

('              precision    recall  f1-score   support\n\n        bird       0.88      0.86      0.87      1000\n       truck       0.86      0.88      0.87      1000\n\n    accuracy                           0.87      2000\n   macro avg       0.87      0.87      0.87      2000\nweighted avg       0.87      0.87      0.87      2000\n',
 '              precision    recall  f1-score   support\n\n        bird       0.90      0.89      0.89      5000\n       truck       0.89      0.90      0.89      5000\n\n    accuracy                           0.89     10000\n   macro avg       0.89      0.89      0.89     10000\nweighted avg       0.89      0.89      0.89     10000\n')

In [15]:
evaluate([3,4])



=== Evaluating classes [3, 4] ===
Selected classes: ['cat', 'deer']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.8167
Epoch [1/10] Batch [200/313] Loss: -0.1661
Epoch [1/10] Batch [300/313] Loss: -0.6977
Epoch [1/10], Average Loss: -0.5323
Epoch [2/10] Batch [100/313] Loss: -0.7861
Epoch [2/10] Batch [200/313] Loss: -0.6712
Epoch [2/10] Batch [300/313] Loss: -0.7563
Epoch [2/10], Average Loss: -0.6634
Epoch [3/10] Batch [100/313] Loss: -1.0840
Epoch [3/10] Batch [200/313] Loss: -0.8086
Epoch [3/10] Batch [300/313] Loss: -0.8581
Epoch [3/10], Average Loss: -0.8783
Epoch [4/10] Batch [100/313] Loss: -1.0617
Epoch [4/10] Batch [200/313] Loss: -0.9979
Epoch [4/10] Batch [300/313] Loss: -0.9061
Epoch [4/10], Average Loss: -1.0903
Epoch [5/10] Batch [100/313] Loss: -1.2342
Epoch [5/10] Batch [200/313] Loss: -1.4245
Epoch [5/10] Batch [300/313] Loss: -1.3771
Epoch [5/10], Average Loss: -1.2490
Epoch [6/10] Batch [100/313] Loss: -1.4566
Epoch [6/10] Batch [200/313] Loss: -1.5373
Epoch [6/10] Batch [300/313] Loss: -1.5535
Epoch [6/10], Average Loss: -1.4331
Epoch [7/1

('              precision    recall  f1-score   support\n\n         cat       0.76      0.60      0.67      1000\n        deer       0.67      0.81      0.73      1000\n\n    accuracy                           0.71      2000\n   macro avg       0.72      0.71      0.70      2000\nweighted avg       0.72      0.71      0.70      2000\n',
 '              precision    recall  f1-score   support\n\n         cat       0.77      0.63      0.69      5000\n        deer       0.69      0.81      0.74      5000\n\n    accuracy                           0.72     10000\n   macro avg       0.73      0.72      0.72     10000\nweighted avg       0.73      0.72      0.72     10000\n')

In [16]:
evaluate([3,5])



=== Evaluating classes [3, 5] ===
Selected classes: ['cat', 'dog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.5063
Epoch [1/10] Batch [200/313] Loss: -0.4263
Epoch [1/10] Batch [300/313] Loss: -0.5257
Epoch [1/10], Average Loss: -0.5053
Epoch [2/10] Batch [100/313] Loss: -0.5580
Epoch [2/10] Batch [200/313] Loss: -0.4141
Epoch [2/10] Batch [300/313] Loss: -0.4827
Epoch [2/10], Average Loss: -0.5126
Epoch [3/10] Batch [100/313] Loss: -0.5014
Epoch [3/10] Batch [200/313] Loss: -0.5241
Epoch [3/10] Batch [300/313] Loss: -0.4226
Epoch [3/10], Average Loss: -0.5025
Epoch [4/10] Batch [100/313] Loss: -0.2952
Epoch [4/10] Batch [200/313] Loss: -0.5072
Epoch [4/10] Batch [300/313] Loss: -0.5354
Epoch [4/10], Average Loss: -0.5134
Epoch [5/10] Batch [100/313] Loss: -0.5897
Epoch [5/10] Batch [200/313] Loss: -0.5220
Epoch [5/10] Batch [300/313] Loss: -0.3448
Epoch [5/10], Average Loss: -0.5210
Epoch [6/10] Batch [100/313] Loss: -0.4085
Epoch [6/10] Batch [200/313] Loss: -0.5576
Epoch [6/10] Batch [300/313] Loss: -0.5838
Epoch [6/10], Average Loss: -0.5155
Epoch [7/1

('              precision    recall  f1-score   support\n\n         cat       0.49      0.35      0.41      1000\n         dog       0.49      0.63      0.55      1000\n\n    accuracy                           0.49      2000\n   macro avg       0.49      0.49      0.48      2000\nweighted avg       0.49      0.49      0.48      2000\n',
 '              precision    recall  f1-score   support\n\n         cat       0.49      0.34      0.40      5000\n         dog       0.49      0.64      0.56      5000\n\n    accuracy                           0.49     10000\n   macro avg       0.49      0.49      0.48     10000\nweighted avg       0.49      0.49      0.48     10000\n')

In [17]:
evaluate([3,6])



=== Evaluating classes [3, 6] ===
Selected classes: ['cat', 'frog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.4942
Epoch [1/10] Batch [200/313] Loss: -0.5365
Epoch [1/10] Batch [300/313] Loss: -0.7211
Epoch [1/10], Average Loss: -0.5546
Epoch [2/10] Batch [100/313] Loss: -0.7901
Epoch [2/10] Batch [200/313] Loss: -0.8248
Epoch [2/10] Batch [300/313] Loss: -0.7404
Epoch [2/10], Average Loss: -0.7006
Epoch [3/10] Batch [100/313] Loss: -0.7473
Epoch [3/10] Batch [200/313] Loss: -1.1265
Epoch [3/10] Batch [300/313] Loss: -1.1072
Epoch [3/10], Average Loss: -0.8690
Epoch [4/10] Batch [100/313] Loss: -1.4459
Epoch [4/10] Batch [200/313] Loss: -0.9652
Epoch [4/10] Batch [300/313] Loss: -0.7526
Epoch [4/10], Average Loss: -1.0218
Epoch [5/10] Batch [100/313] Loss: -1.3351
Epoch [5/10] Batch [200/313] Loss: -1.4291
Epoch [5/10] Batch [300/313] Loss: -1.7897
Epoch [5/10], Average Loss: -1.1748
Epoch [6/10] Batch [100/313] Loss: -1.1765
Epoch [6/10] Batch [200/313] Loss: -0.9584
Epoch [6/10] Batch [300/313] Loss: -1.3666
Epoch [6/10], Average Loss: -1.3132
Epoch [7/1

('              precision    recall  f1-score   support\n\n         cat       0.68      0.76      0.72      1000\n        frog       0.73      0.65      0.68      1000\n\n    accuracy                           0.70      2000\n   macro avg       0.71      0.70      0.70      2000\nweighted avg       0.71      0.70      0.70      2000\n',
 '              precision    recall  f1-score   support\n\n         cat       0.69      0.76      0.72      5000\n        frog       0.74      0.66      0.69      5000\n\n    accuracy                           0.71     10000\n   macro avg       0.71      0.71      0.71     10000\nweighted avg       0.71      0.71      0.71     10000\n')

In [18]:
evaluate([3,7])



=== Evaluating classes [3, 7] ===
Selected classes: ['cat', 'horse']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.4308
Epoch [1/10] Batch [200/313] Loss: -0.3761
Epoch [1/10] Batch [300/313] Loss: -0.6392
Epoch [1/10], Average Loss: -0.5251
Epoch [2/10] Batch [100/313] Loss: -0.4498
Epoch [2/10] Batch [200/313] Loss: -0.4705
Epoch [2/10] Batch [300/313] Loss: -0.9003
Epoch [2/10], Average Loss: -0.6360
Epoch [3/10] Batch [100/313] Loss: -0.5250
Epoch [3/10] Batch [200/313] Loss: -0.8817
Epoch [3/10] Batch [300/313] Loss: -0.7641
Epoch [3/10], Average Loss: -0.8595
Epoch [4/10] Batch [100/313] Loss: -1.2005
Epoch [4/10] Batch [200/313] Loss: -1.3486
Epoch [4/10] Batch [300/313] Loss: -1.0219
Epoch [4/10], Average Loss: -1.1113
Epoch [5/10] Batch [100/313] Loss: -1.3106
Epoch [5/10] Batch [200/313] Loss: -1.1546
Epoch [5/10] Batch [300/313] Loss: -0.8319
Epoch [5/10], Average Loss: -1.3125
Epoch [6/10] Batch [100/313] Loss: -1.0914
Epoch [6/10] Batch [200/313] Loss: -1.4568
Epoch [6/10] Batch [300/313] Loss: -1.1949
Epoch [6/10], Average Loss: -1.5260
Epoch [7/1

('              precision    recall  f1-score   support\n\n         cat       0.79      0.61      0.69      1000\n       horse       0.68      0.84      0.75      1000\n\n    accuracy                           0.72      2000\n   macro avg       0.74      0.72      0.72      2000\nweighted avg       0.74      0.72      0.72      2000\n',
 '              precision    recall  f1-score   support\n\n         cat       0.82      0.67      0.74      5000\n       horse       0.72      0.86      0.78      5000\n\n    accuracy                           0.76     10000\n   macro avg       0.77      0.76      0.76     10000\nweighted avg       0.77      0.76      0.76     10000\n')

In [19]:
evaluate([3,8])



=== Evaluating classes [3, 8] ===
Selected classes: ['cat', 'ship']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6063
Epoch [1/10] Batch [200/313] Loss: -0.7114
Epoch [1/10] Batch [300/313] Loss: -0.8125
Epoch [1/10], Average Loss: -0.6871
Epoch [2/10] Batch [100/313] Loss: -1.0141
Epoch [2/10] Batch [200/313] Loss: -1.1032
Epoch [2/10] Batch [300/313] Loss: -1.0887
Epoch [2/10], Average Loss: -0.9419
Epoch [3/10] Batch [100/313] Loss: -1.0686
Epoch [3/10] Batch [200/313] Loss: -1.2213
Epoch [3/10] Batch [300/313] Loss: -1.3934
Epoch [3/10], Average Loss: -1.2024
Epoch [4/10] Batch [100/313] Loss: -1.3666
Epoch [4/10] Batch [200/313] Loss: -1.1149
Epoch [4/10] Batch [300/313] Loss: -1.8342
Epoch [4/10], Average Loss: -1.4644
Epoch [5/10] Batch [100/313] Loss: -1.4655
Epoch [5/10] Batch [200/313] Loss: -1.5200
Epoch [5/10] Batch [300/313] Loss: -1.7059
Epoch [5/10], Average Loss: -1.7113
Epoch [6/10] Batch [100/313] Loss: -1.8159
Epoch [6/10] Batch [200/313] Loss: -2.0864
Epoch [6/10] Batch [300/313] Loss: -2.5282
Epoch [6/10], Average Loss: -1.9634
Epoch [7/1

('              precision    recall  f1-score   support\n\n         cat       0.89      0.87      0.88      1000\n        ship       0.87      0.89      0.88      1000\n\n    accuracy                           0.88      2000\n   macro avg       0.88      0.88      0.88      2000\nweighted avg       0.88      0.88      0.88      2000\n',
 '              precision    recall  f1-score   support\n\n         cat       0.88      0.89      0.88      5000\n        ship       0.88      0.88      0.88      5000\n\n    accuracy                           0.88     10000\n   macro avg       0.88      0.88      0.88     10000\nweighted avg       0.88      0.88      0.88     10000\n')

In [20]:
evaluate([3,9])



=== Evaluating classes [3, 9] ===
Selected classes: ['cat', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.7485
Epoch [1/10] Batch [200/313] Loss: -0.7518
Epoch [1/10] Batch [300/313] Loss: -0.8109
Epoch [1/10], Average Loss: -0.8093
Epoch [2/10] Batch [100/313] Loss: -1.1341
Epoch [2/10] Batch [200/313] Loss: -0.9674
Epoch [2/10] Batch [300/313] Loss: -1.3052
Epoch [2/10], Average Loss: -1.1420
Epoch [3/10] Batch [100/313] Loss: -1.3257
Epoch [3/10] Batch [200/313] Loss: -1.3748
Epoch [3/10] Batch [300/313] Loss: -1.5811
Epoch [3/10], Average Loss: -1.4162
Epoch [4/10] Batch [100/313] Loss: -1.4170
Epoch [4/10] Batch [200/313] Loss: -1.4049
Epoch [4/10] Batch [300/313] Loss: -1.8542
Epoch [4/10], Average Loss: -1.6970
Epoch [5/10] Batch [100/313] Loss: -2.0426
Epoch [5/10] Batch [200/313] Loss: -1.5577
Epoch [5/10] Batch [300/313] Loss: -2.7406
Epoch [5/10], Average Loss: -1.9240
Epoch [6/10] Batch [100/313] Loss: -2.3392
Epoch [6/10] Batch [200/313] Loss: -2.2555
Epoch [6/10] Batch [300/313] Loss: -2.1500
Epoch [6/10], Average Loss: -2.1898
Epoch [7/1

('              precision    recall  f1-score   support\n\n         cat       0.84      0.89      0.87      1000\n       truck       0.89      0.83      0.86      1000\n\n    accuracy                           0.86      2000\n   macro avg       0.86      0.86      0.86      2000\nweighted avg       0.86      0.86      0.86      2000\n',
 '              precision    recall  f1-score   support\n\n         cat       0.85      0.90      0.88      5000\n       truck       0.90      0.85      0.87      5000\n\n    accuracy                           0.87     10000\n   macro avg       0.88      0.87      0.87     10000\nweighted avg       0.88      0.87      0.87     10000\n')

In [21]:
evaluate([4,5])



=== Evaluating classes [4, 5] ===
Selected classes: ['deer', 'dog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.4432
Epoch [1/10] Batch [200/313] Loss: -0.9251
Epoch [1/10] Batch [300/313] Loss: -0.7042
Epoch [1/10], Average Loss: -0.5322
Epoch [2/10] Batch [100/313] Loss: -0.7429
Epoch [2/10] Batch [200/313] Loss: -0.8222
Epoch [2/10] Batch [300/313] Loss: -0.7993
Epoch [2/10], Average Loss: -0.6839
Epoch [3/10] Batch [100/313] Loss: -0.4187
Epoch [3/10] Batch [200/313] Loss: -0.7891
Epoch [3/10] Batch [300/313] Loss: -1.0461
Epoch [3/10], Average Loss: -0.8472
Epoch [4/10] Batch [100/313] Loss: -1.0684
Epoch [4/10] Batch [200/313] Loss: -1.3004
Epoch [4/10] Batch [300/313] Loss: -0.9368
Epoch [4/10], Average Loss: -1.0167
Epoch [5/10] Batch [100/313] Loss: -1.1078
Epoch [5/10] Batch [200/313] Loss: -1.2736
Epoch [5/10] Batch [300/313] Loss: -1.3098
Epoch [5/10], Average Loss: -1.1795
Epoch [6/10] Batch [100/313] Loss: -1.0618
Epoch [6/10] Batch [200/313] Loss: -1.4937
Epoch [6/10] Batch [300/313] Loss: -1.1129
Epoch [6/10], Average Loss: -1.3375
Epoch [7/1

('              precision    recall  f1-score   support\n\n        deer       0.73      0.76      0.74      1000\n         dog       0.75      0.72      0.74      1000\n\n    accuracy                           0.74      2000\n   macro avg       0.74      0.74      0.74      2000\nweighted avg       0.74      0.74      0.74      2000\n',
 '              precision    recall  f1-score   support\n\n        deer       0.77      0.81      0.79      5000\n         dog       0.80      0.76      0.78      5000\n\n    accuracy                           0.78     10000\n   macro avg       0.78      0.78      0.78     10000\nweighted avg       0.78      0.78      0.78     10000\n')

In [22]:
evaluate([4,6])



=== Evaluating classes [4, 6] ===
Selected classes: ['deer', 'frog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.5159
Epoch [1/10] Batch [200/313] Loss: -0.5449
Epoch [1/10] Batch [300/313] Loss: -0.6477
Epoch [1/10], Average Loss: -0.5403
Epoch [2/10] Batch [100/313] Loss: -0.6427
Epoch [2/10] Batch [200/313] Loss: -0.7739
Epoch [2/10] Batch [300/313] Loss: -0.7670
Epoch [2/10], Average Loss: -0.7031
Epoch [3/10] Batch [100/313] Loss: -0.8589
Epoch [3/10] Batch [200/313] Loss: -0.8105
Epoch [3/10] Batch [300/313] Loss: -0.8828
Epoch [3/10], Average Loss: -0.8874
Epoch [4/10] Batch [100/313] Loss: -0.7968
Epoch [4/10] Batch [200/313] Loss: -1.0958
Epoch [4/10] Batch [300/313] Loss: -1.2754
Epoch [4/10], Average Loss: -1.0689
Epoch [5/10] Batch [100/313] Loss: -1.3641
Epoch [5/10] Batch [200/313] Loss: -1.7283
Epoch [5/10] Batch [300/313] Loss: -1.3635
Epoch [5/10], Average Loss: -1.2639
Epoch [6/10] Batch [100/313] Loss: -1.4583
Epoch [6/10] Batch [200/313] Loss: -1.4951
Epoch [6/10] Batch [300/313] Loss: -1.2990
Epoch [6/10], Average Loss: -1.4166
Epoch [7/1

('              precision    recall  f1-score   support\n\n        deer       0.81      0.69      0.75      1000\n        frog       0.73      0.84      0.78      1000\n\n    accuracy                           0.77      2000\n   macro avg       0.77      0.76      0.76      2000\nweighted avg       0.77      0.77      0.76      2000\n',
 '              precision    recall  f1-score   support\n\n        deer       0.83      0.73      0.78      5000\n        frog       0.76      0.85      0.80      5000\n\n    accuracy                           0.79     10000\n   macro avg       0.80      0.79      0.79     10000\nweighted avg       0.80      0.79      0.79     10000\n')

In [23]:
evaluate([4,7])



=== Evaluating classes [4, 7] ===
Selected classes: ['deer', 'horse']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.3516
Epoch [1/10] Batch [200/313] Loss: -0.3189
Epoch [1/10] Batch [300/313] Loss: -0.6119
Epoch [1/10], Average Loss: -0.5144
Epoch [2/10] Batch [100/313] Loss: -0.4770
Epoch [2/10] Batch [200/313] Loss: -0.4291
Epoch [2/10] Batch [300/313] Loss: -0.7616
Epoch [2/10], Average Loss: -0.6206
Epoch [3/10] Batch [100/313] Loss: -0.6059
Epoch [3/10] Batch [200/313] Loss: -0.7487
Epoch [3/10] Batch [300/313] Loss: -0.7498
Epoch [3/10], Average Loss: -0.7712
Epoch [4/10] Batch [100/313] Loss: -1.1818
Epoch [4/10] Batch [200/313] Loss: -0.2863
Epoch [4/10] Batch [300/313] Loss: -0.7004
Epoch [4/10], Average Loss: -0.9162
Epoch [5/10] Batch [100/313] Loss: -1.1741
Epoch [5/10] Batch [200/313] Loss: -1.3238
Epoch [5/10] Batch [300/313] Loss: -1.2784
Epoch [5/10], Average Loss: -1.0683
Epoch [6/10] Batch [100/313] Loss: -1.1196
Epoch [6/10] Batch [200/313] Loss: -1.6742
Epoch [6/10] Batch [300/313] Loss: -1.2145
Epoch [6/10], Average Loss: -1.2213
Epoch [7/1

('              precision    recall  f1-score   support\n\n        deer       0.56      0.96      0.71      1000\n       horse       0.86      0.26      0.40      1000\n\n    accuracy                           0.61      2000\n   macro avg       0.71      0.61      0.55      2000\nweighted avg       0.71      0.61      0.55      2000\n',
 '              precision    recall  f1-score   support\n\n        deer       0.56      0.95      0.70      5000\n       horse       0.84      0.24      0.38      5000\n\n    accuracy                           0.60     10000\n   macro avg       0.70      0.60      0.54     10000\nweighted avg       0.70      0.60      0.54     10000\n')

In [24]:
evaluate([4,8])



=== Evaluating classes [4, 8] ===
Selected classes: ['deer', 'ship']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6304
Epoch [1/10] Batch [200/313] Loss: -0.8899
Epoch [1/10] Batch [300/313] Loss: -1.1888
Epoch [1/10], Average Loss: -0.7792
Epoch [2/10] Batch [100/313] Loss: -0.9233
Epoch [2/10] Batch [200/313] Loss: -0.9735
Epoch [2/10] Batch [300/313] Loss: -1.1095
Epoch [2/10], Average Loss: -1.1540
Epoch [3/10] Batch [100/313] Loss: -1.3577
Epoch [3/10] Batch [200/313] Loss: -1.3835
Epoch [3/10] Batch [300/313] Loss: -1.6716
Epoch [3/10], Average Loss: -1.4345
Epoch [4/10] Batch [100/313] Loss: -1.2252
Epoch [4/10] Batch [200/313] Loss: -1.5829
Epoch [4/10] Batch [300/313] Loss: -1.6844
Epoch [4/10], Average Loss: -1.7516
Epoch [5/10] Batch [100/313] Loss: -2.2944
Epoch [5/10] Batch [200/313] Loss: -2.2431
Epoch [5/10] Batch [300/313] Loss: -2.3088
Epoch [5/10], Average Loss: -2.0493
Epoch [6/10] Batch [100/313] Loss: -2.5044
Epoch [6/10] Batch [200/313] Loss: -1.9484
Epoch [6/10] Batch [300/313] Loss: -2.7709
Epoch [6/10], Average Loss: -2.3384
Epoch [7/1

('              precision    recall  f1-score   support\n\n        deer       0.87      0.87      0.87      1000\n        ship       0.87      0.87      0.87      1000\n\n    accuracy                           0.87      2000\n   macro avg       0.87      0.87      0.87      2000\nweighted avg       0.87      0.87      0.87      2000\n',
 '              precision    recall  f1-score   support\n\n        deer       0.87      0.86      0.86      5000\n        ship       0.86      0.87      0.86      5000\n\n    accuracy                           0.86     10000\n   macro avg       0.86      0.86      0.86     10000\nweighted avg       0.86      0.86      0.86     10000\n')

In [25]:
evaluate([4,9])



=== Evaluating classes [4, 9] ===
Selected classes: ['deer', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6052
Epoch [1/10] Batch [200/313] Loss: -0.6641
Epoch [1/10] Batch [300/313] Loss: -0.8407
Epoch [1/10], Average Loss: -0.6590
Epoch [2/10] Batch [100/313] Loss: -0.9839
Epoch [2/10] Batch [200/313] Loss: -1.1586
Epoch [2/10] Batch [300/313] Loss: -1.2516
Epoch [2/10], Average Loss: -0.9985
Epoch [3/10] Batch [100/313] Loss: -0.9437
Epoch [3/10] Batch [200/313] Loss: -1.3424
Epoch [3/10] Batch [300/313] Loss: -1.3748
Epoch [3/10], Average Loss: -1.3095
Epoch [4/10] Batch [100/313] Loss: -1.4087
Epoch [4/10] Batch [200/313] Loss: -1.5060
Epoch [4/10] Batch [300/313] Loss: -1.9213
Epoch [4/10], Average Loss: -1.5943
Epoch [5/10] Batch [100/313] Loss: -1.8996
Epoch [5/10] Batch [200/313] Loss: -1.6198
Epoch [5/10] Batch [300/313] Loss: -2.1951
Epoch [5/10], Average Loss: -1.8495
Epoch [6/10] Batch [100/313] Loss: -1.2725
Epoch [6/10] Batch [200/313] Loss: -2.1408
Epoch [6/10] Batch [300/313] Loss: -1.9108
Epoch [6/10], Average Loss: -2.1011
Epoch [7/1

('              precision    recall  f1-score   support\n\n        deer       0.86      0.93      0.89      1000\n       truck       0.92      0.85      0.89      1000\n\n    accuracy                           0.89      2000\n   macro avg       0.89      0.89      0.89      2000\nweighted avg       0.89      0.89      0.89      2000\n',
 '              precision    recall  f1-score   support\n\n        deer       0.88      0.91      0.90      5000\n       truck       0.91      0.88      0.89      5000\n\n    accuracy                           0.89     10000\n   macro avg       0.90      0.89      0.89     10000\nweighted avg       0.90      0.89      0.89     10000\n')

In [26]:
evaluate([5,6])



=== Evaluating classes [5, 6] ===
Selected classes: ['dog', 'frog']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -1.0095
Epoch [1/10] Batch [200/313] Loss: -0.8848
Epoch [1/10] Batch [300/313] Loss: -0.5081
Epoch [1/10], Average Loss: -0.6767
Epoch [2/10] Batch [100/313] Loss: -0.9289
Epoch [2/10] Batch [200/313] Loss: -0.6864
Epoch [2/10] Batch [300/313] Loss: -1.0458
Epoch [2/10], Average Loss: -0.8881
Epoch [3/10] Batch [100/313] Loss: -0.8749
Epoch [3/10] Batch [200/313] Loss: -1.1691
Epoch [3/10] Batch [300/313] Loss: -1.0994
Epoch [3/10], Average Loss: -1.0906
Epoch [4/10] Batch [100/313] Loss: -1.6546
Epoch [4/10] Batch [200/313] Loss: -1.2304
Epoch [4/10] Batch [300/313] Loss: -0.8791
Epoch [4/10], Average Loss: -1.2788
Epoch [5/10] Batch [100/313] Loss: -1.3957
Epoch [5/10] Batch [200/313] Loss: -1.6841
Epoch [5/10] Batch [300/313] Loss: -1.2554
Epoch [5/10], Average Loss: -1.4702
Epoch [6/10] Batch [100/313] Loss: -1.7067
Epoch [6/10] Batch [200/313] Loss: -1.6451
Epoch [6/10] Batch [300/313] Loss: -0.8109
Epoch [6/10], Average Loss: -1.6516
Epoch [7/1

('              precision    recall  f1-score   support\n\n         dog       0.78      0.79      0.79      1000\n        frog       0.79      0.78      0.78      1000\n\n    accuracy                           0.79      2000\n   macro avg       0.79      0.79      0.79      2000\nweighted avg       0.79      0.79      0.79      2000\n',
 '              precision    recall  f1-score   support\n\n         dog       0.81      0.79      0.80      5000\n        frog       0.80      0.81      0.80      5000\n\n    accuracy                           0.80     10000\n   macro avg       0.80      0.80      0.80     10000\nweighted avg       0.80      0.80      0.80     10000\n')

In [27]:
evaluate([5,7])



=== Evaluating classes [5, 7] ===
Selected classes: ['dog', 'horse']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.4351
Epoch [1/10] Batch [200/313] Loss: -0.3192
Epoch [1/10] Batch [300/313] Loss: -0.7439
Epoch [1/10], Average Loss: -0.5354
Epoch [2/10] Batch [100/313] Loss: -0.4343
Epoch [2/10] Batch [200/313] Loss: -0.6512
Epoch [2/10] Batch [300/313] Loss: -0.4243
Epoch [2/10], Average Loss: -0.6534
Epoch [3/10] Batch [100/313] Loss: -0.8465
Epoch [3/10] Batch [200/313] Loss: -0.7390
Epoch [3/10] Batch [300/313] Loss: -0.6570
Epoch [3/10], Average Loss: -0.7986
Epoch [4/10] Batch [100/313] Loss: -0.6154
Epoch [4/10] Batch [200/313] Loss: -0.7567
Epoch [4/10] Batch [300/313] Loss: -1.3140
Epoch [4/10], Average Loss: -0.9561
Epoch [5/10] Batch [100/313] Loss: -1.4667
Epoch [5/10] Batch [200/313] Loss: -1.4275
Epoch [5/10] Batch [300/313] Loss: -0.9431
Epoch [5/10], Average Loss: -1.1439
Epoch [6/10] Batch [100/313] Loss: -1.2643
Epoch [6/10] Batch [200/313] Loss: -1.2929
Epoch [6/10] Batch [300/313] Loss: -1.3667
Epoch [6/10], Average Loss: -1.3029
Epoch [7/1

('              precision    recall  f1-score   support\n\n         dog       0.80      0.64      0.71      1000\n       horse       0.70      0.84      0.76      1000\n\n    accuracy                           0.74      2000\n   macro avg       0.75      0.74      0.74      2000\nweighted avg       0.75      0.74      0.74      2000\n',
 '              precision    recall  f1-score   support\n\n         dog       0.83      0.68      0.75      5000\n       horse       0.73      0.86      0.79      5000\n\n    accuracy                           0.77     10000\n   macro avg       0.78      0.77      0.77     10000\nweighted avg       0.78      0.77      0.77     10000\n')

In [28]:
evaluate([5,8])



=== Evaluating classes [5, 8] ===
Selected classes: ['dog', 'ship']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.4854
Epoch [1/10] Batch [200/313] Loss: -0.7531
Epoch [1/10] Batch [300/313] Loss: -0.5997
Epoch [1/10], Average Loss: -0.5917
Epoch [2/10] Batch [100/313] Loss: -1.0545
Epoch [2/10] Batch [200/313] Loss: -1.3157
Epoch [2/10] Batch [300/313] Loss: -1.0781
Epoch [2/10], Average Loss: -0.9061
Epoch [3/10] Batch [100/313] Loss: -1.2814
Epoch [3/10] Batch [200/313] Loss: -1.2832
Epoch [3/10] Batch [300/313] Loss: -1.3103
Epoch [3/10], Average Loss: -1.2393
Epoch [4/10] Batch [100/313] Loss: -1.4066
Epoch [4/10] Batch [200/313] Loss: -1.3442
Epoch [4/10] Batch [300/313] Loss: -1.7668
Epoch [4/10], Average Loss: -1.5301
Epoch [5/10] Batch [100/313] Loss: -2.2056
Epoch [5/10] Batch [200/313] Loss: -1.8731
Epoch [5/10] Batch [300/313] Loss: -2.2786
Epoch [5/10], Average Loss: -1.8101
Epoch [6/10] Batch [100/313] Loss: -2.1193
Epoch [6/10] Batch [200/313] Loss: -2.0401
Epoch [6/10] Batch [300/313] Loss: -2.1030
Epoch [6/10], Average Loss: -2.0987
Epoch [7/1

('              precision    recall  f1-score   support\n\n         dog       0.91      0.94      0.92      1000\n        ship       0.94      0.90      0.92      1000\n\n    accuracy                           0.92      2000\n   macro avg       0.92      0.92      0.92      2000\nweighted avg       0.92      0.92      0.92      2000\n',
 '              precision    recall  f1-score   support\n\n         dog       0.91      0.95      0.93      5000\n        ship       0.95      0.91      0.93      5000\n\n    accuracy                           0.93     10000\n   macro avg       0.93      0.93      0.93     10000\nweighted avg       0.93      0.93      0.93     10000\n')

In [29]:
evaluate([5,9])



=== Evaluating classes [5, 9] ===
Selected classes: ['dog', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.5346
Epoch [1/10] Batch [200/313] Loss: -0.7031
Epoch [1/10] Batch [300/313] Loss: -0.7047
Epoch [1/10], Average Loss: -0.6274
Epoch [2/10] Batch [100/313] Loss: -0.7402
Epoch [2/10] Batch [200/313] Loss: -0.9159
Epoch [2/10] Batch [300/313] Loss: -0.9705
Epoch [2/10], Average Loss: -0.8478
Epoch [3/10] Batch [100/313] Loss: -1.0835
Epoch [3/10] Batch [200/313] Loss: -1.2453
Epoch [3/10] Batch [300/313] Loss: -1.0698
Epoch [3/10], Average Loss: -1.0736
Epoch [4/10] Batch [100/313] Loss: -1.3624
Epoch [4/10] Batch [200/313] Loss: -1.5258
Epoch [4/10] Batch [300/313] Loss: -1.5056
Epoch [4/10], Average Loss: -1.3193
Epoch [5/10] Batch [100/313] Loss: -1.3910
Epoch [5/10] Batch [200/313] Loss: -1.9055
Epoch [5/10] Batch [300/313] Loss: -1.8154
Epoch [5/10], Average Loss: -1.5421
Epoch [6/10] Batch [100/313] Loss: -1.6037
Epoch [6/10] Batch [200/313] Loss: -2.0890
Epoch [6/10] Batch [300/313] Loss: -1.8771
Epoch [6/10], Average Loss: -1.7811
Epoch [7/1

('              precision    recall  f1-score   support\n\n         dog       0.82      0.94      0.88      1000\n       truck       0.93      0.79      0.86      1000\n\n    accuracy                           0.87      2000\n   macro avg       0.87      0.87      0.87      2000\nweighted avg       0.87      0.87      0.87      2000\n',
 '              precision    recall  f1-score   support\n\n         dog       0.83      0.94      0.88      5000\n       truck       0.93      0.80      0.86      5000\n\n    accuracy                           0.87     10000\n   macro avg       0.88      0.87      0.87     10000\nweighted avg       0.88      0.87      0.87     10000\n')

In [30]:
evaluate([6,7])



=== Evaluating classes [6, 7] ===
Selected classes: ['frog', 'horse']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6609
Epoch [1/10] Batch [200/313] Loss: -0.8958
Epoch [1/10] Batch [300/313] Loss: -1.0081
Epoch [1/10], Average Loss: -0.7071
Epoch [2/10] Batch [100/313] Loss: -1.0686
Epoch [2/10] Batch [200/313] Loss: -1.0600
Epoch [2/10] Batch [300/313] Loss: -1.0375
Epoch [2/10], Average Loss: -0.9639
Epoch [3/10] Batch [100/313] Loss: -1.1264
Epoch [3/10] Batch [200/313] Loss: -1.1218
Epoch [3/10] Batch [300/313] Loss: -1.0738
Epoch [3/10], Average Loss: -1.2175
Epoch [4/10] Batch [100/313] Loss: -1.5692
Epoch [4/10] Batch [200/313] Loss: -1.4879
Epoch [4/10] Batch [300/313] Loss: -1.5796
Epoch [4/10], Average Loss: -1.4764
Epoch [5/10] Batch [100/313] Loss: -1.2559
Epoch [5/10] Batch [200/313] Loss: -1.4360
Epoch [5/10] Batch [300/313] Loss: -2.1374
Epoch [5/10], Average Loss: -1.7396
Epoch [6/10] Batch [100/313] Loss: -2.1882
Epoch [6/10] Batch [200/313] Loss: -2.2997
Epoch [6/10] Batch [300/313] Loss: -1.9866
Epoch [6/10], Average Loss: -1.9914
Epoch [7/1

('              precision    recall  f1-score   support\n\n        frog       0.86      0.73      0.79      1000\n       horse       0.77      0.88      0.82      1000\n\n    accuracy                           0.81      2000\n   macro avg       0.82      0.81      0.81      2000\nweighted avg       0.82      0.81      0.81      2000\n',
 '              precision    recall  f1-score   support\n\n        frog       0.87      0.74      0.80      5000\n       horse       0.77      0.89      0.83      5000\n\n    accuracy                           0.81     10000\n   macro avg       0.82      0.81      0.81     10000\nweighted avg       0.82      0.81      0.81     10000\n')

In [31]:
evaluate([6,8])



=== Evaluating classes [6, 8] ===
Selected classes: ['frog', 'ship']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.7411
Epoch [1/10] Batch [200/313] Loss: -0.9070
Epoch [1/10] Batch [300/313] Loss: -0.7646
Epoch [1/10], Average Loss: -0.6029
Epoch [2/10] Batch [100/313] Loss: -0.8098
Epoch [2/10] Batch [200/313] Loss: -0.8396
Epoch [2/10] Batch [300/313] Loss: -1.2391
Epoch [2/10], Average Loss: -0.8740
Epoch [3/10] Batch [100/313] Loss: -1.4234
Epoch [3/10] Batch [200/313] Loss: -1.2327
Epoch [3/10] Batch [300/313] Loss: -1.4500
Epoch [3/10], Average Loss: -1.1383
Epoch [4/10] Batch [100/313] Loss: -1.3395
Epoch [4/10] Batch [200/313] Loss: -1.3327
Epoch [4/10] Batch [300/313] Loss: -1.1613
Epoch [4/10], Average Loss: -1.3846
Epoch [5/10] Batch [100/313] Loss: -1.4928
Epoch [5/10] Batch [200/313] Loss: -1.8813
Epoch [5/10] Batch [300/313] Loss: -1.7059
Epoch [5/10], Average Loss: -1.6446
Epoch [6/10] Batch [100/313] Loss: -1.7730
Epoch [6/10] Batch [200/313] Loss: -1.6649
Epoch [6/10] Batch [300/313] Loss: -2.0385
Epoch [6/10], Average Loss: -1.9028
Epoch [7/1

('              precision    recall  f1-score   support\n\n        frog       0.92      0.83      0.87      1000\n        ship       0.85      0.92      0.88      1000\n\n    accuracy                           0.88      2000\n   macro avg       0.88      0.88      0.88      2000\nweighted avg       0.88      0.88      0.88      2000\n',
 '              precision    recall  f1-score   support\n\n        frog       0.93      0.85      0.89      5000\n        ship       0.86      0.93      0.90      5000\n\n    accuracy                           0.89     10000\n   macro avg       0.89      0.89      0.89     10000\nweighted avg       0.89      0.89      0.89     10000\n')

In [32]:
evaluate([6,9])



=== Evaluating classes [6, 9] ===
Selected classes: ['frog', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6029
Epoch [1/10] Batch [200/313] Loss: -0.7375
Epoch [1/10] Batch [300/313] Loss: -0.7231
Epoch [1/10], Average Loss: -0.7192
Epoch [2/10] Batch [100/313] Loss: -0.8982
Epoch [2/10] Batch [200/313] Loss: -1.0058
Epoch [2/10] Batch [300/313] Loss: -1.2510
Epoch [2/10], Average Loss: -0.9981
Epoch [3/10] Batch [100/313] Loss: -0.9568
Epoch [3/10] Batch [200/313] Loss: -1.5331
Epoch [3/10] Batch [300/313] Loss: -1.5275
Epoch [3/10], Average Loss: -1.2653
Epoch [4/10] Batch [100/313] Loss: -1.4315
Epoch [4/10] Batch [200/313] Loss: -1.3644
Epoch [4/10] Batch [300/313] Loss: -1.8838
Epoch [4/10], Average Loss: -1.5258
Epoch [5/10] Batch [100/313] Loss: -1.5035
Epoch [5/10] Batch [200/313] Loss: -1.9058
Epoch [5/10] Batch [300/313] Loss: -2.1828
Epoch [5/10], Average Loss: -1.7888
Epoch [6/10] Batch [100/313] Loss: -2.0794
Epoch [6/10] Batch [200/313] Loss: -1.5438
Epoch [6/10] Batch [300/313] Loss: -2.2289
Epoch [6/10], Average Loss: -2.0109
Epoch [7/1

('              precision    recall  f1-score   support\n\n        frog       0.85      0.87      0.86      1000\n       truck       0.86      0.84      0.85      1000\n\n    accuracy                           0.85      2000\n   macro avg       0.86      0.85      0.85      2000\nweighted avg       0.86      0.85      0.85      2000\n',
 '              precision    recall  f1-score   support\n\n        frog       0.88      0.88      0.88      5000\n       truck       0.88      0.88      0.88      5000\n\n    accuracy                           0.88     10000\n   macro avg       0.88      0.88      0.88     10000\nweighted avg       0.88      0.88      0.88     10000\n')

In [33]:
evaluate([7,8])



=== Evaluating classes [7, 8] ===
Selected classes: ['horse', 'ship']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6949
Epoch [1/10] Batch [200/313] Loss: -0.6272
Epoch [1/10] Batch [300/313] Loss: -0.6317
Epoch [1/10], Average Loss: -0.6082
Epoch [2/10] Batch [100/313] Loss: -0.8184
Epoch [2/10] Batch [200/313] Loss: -0.7696
Epoch [2/10] Batch [300/313] Loss: -1.0743
Epoch [2/10], Average Loss: -0.8869
Epoch [3/10] Batch [100/313] Loss: -1.6257
Epoch [3/10] Batch [200/313] Loss: -1.4495
Epoch [3/10] Batch [300/313] Loss: -1.5278
Epoch [3/10], Average Loss: -1.4177
Epoch [4/10] Batch [100/313] Loss: -1.7014
Epoch [4/10] Batch [200/313] Loss: -1.5602
Epoch [4/10] Batch [300/313] Loss: -1.8401
Epoch [4/10], Average Loss: -1.8049
Epoch [5/10] Batch [100/313] Loss: -2.3949
Epoch [5/10] Batch [200/313] Loss: -1.9428
Epoch [5/10] Batch [300/313] Loss: -2.1295
Epoch [5/10], Average Loss: -2.1072
Epoch [6/10] Batch [100/313] Loss: -2.3307
Epoch [6/10] Batch [200/313] Loss: -2.3791
Epoch [6/10] Batch [300/313] Loss: -2.9086
Epoch [6/10], Average Loss: -2.4113
Epoch [7/1

('              precision    recall  f1-score   support\n\n       horse       0.89      0.91      0.90      1000\n        ship       0.91      0.89      0.90      1000\n\n    accuracy                           0.90      2000\n   macro avg       0.90      0.90      0.90      2000\nweighted avg       0.90      0.90      0.90      2000\n',
 '              precision    recall  f1-score   support\n\n       horse       0.90      0.93      0.91      5000\n        ship       0.92      0.90      0.91      5000\n\n    accuracy                           0.91     10000\n   macro avg       0.91      0.91      0.91     10000\nweighted avg       0.91      0.91      0.91     10000\n')

In [34]:
evaluate([7,9])



=== Evaluating classes [7, 9] ===
Selected classes: ['horse', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6600
Epoch [1/10] Batch [200/313] Loss: -0.8659
Epoch [1/10] Batch [300/313] Loss: -0.4018
Epoch [1/10], Average Loss: -0.6428
Epoch [2/10] Batch [100/313] Loss: -0.8073
Epoch [2/10] Batch [200/313] Loss: -0.7325
Epoch [2/10] Batch [300/313] Loss: -1.3089
Epoch [2/10], Average Loss: -0.9170
Epoch [3/10] Batch [100/313] Loss: -1.2982
Epoch [3/10] Batch [200/313] Loss: -1.3010
Epoch [3/10] Batch [300/313] Loss: -1.7838
Epoch [3/10], Average Loss: -1.1800
Epoch [4/10] Batch [100/313] Loss: -1.0513
Epoch [4/10] Batch [200/313] Loss: -1.4670
Epoch [4/10] Batch [300/313] Loss: -1.8597
Epoch [4/10], Average Loss: -1.4190
Epoch [5/10] Batch [100/313] Loss: -1.6549
Epoch [5/10] Batch [200/313] Loss: -1.7037
Epoch [5/10] Batch [300/313] Loss: -1.9362
Epoch [5/10], Average Loss: -1.6463
Epoch [6/10] Batch [100/313] Loss: -2.1067
Epoch [6/10] Batch [200/313] Loss: -1.9651
Epoch [6/10] Batch [300/313] Loss: -1.7877
Epoch [6/10], Average Loss: -1.8669
Epoch [7/1

('              precision    recall  f1-score   support\n\n       horse       0.85      0.79      0.82      1000\n       truck       0.80      0.86      0.83      1000\n\n    accuracy                           0.82      2000\n   macro avg       0.82      0.82      0.82      2000\nweighted avg       0.82      0.82      0.82      2000\n',
 '              precision    recall  f1-score   support\n\n       horse       0.86      0.81      0.83      5000\n       truck       0.82      0.87      0.84      5000\n\n    accuracy                           0.84     10000\n   macro avg       0.84      0.84      0.84     10000\nweighted avg       0.84      0.84      0.84     10000\n')

In [35]:
evaluate([8,9])



=== Evaluating classes [8, 9] ===
Selected classes: ['ship', 'truck']
Using device: mps


/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:15: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_8588/3371341144.py:16: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch [1/10] Batch [100/313] Loss: -0.6149
Epoch [1/10] Batch [200/313] Loss: -0.6670
Epoch [1/10] Batch [300/313] Loss: -0.5796
Epoch [1/10], Average Loss: -0.5803
Epoch [2/10] Batch [100/313] Loss: -0.6639
Epoch [2/10] Batch [200/313] Loss: -0.7554
Epoch [2/10] Batch [300/313] Loss: -1.0424
Epoch [2/10], Average Loss: -0.8269
Epoch [3/10] Batch [100/313] Loss: -1.1226
Epoch [3/10] Batch [200/313] Loss: -1.1181
Epoch [3/10] Batch [300/313] Loss: -1.0158
Epoch [3/10], Average Loss: -1.1206
Epoch [4/10] Batch [100/313] Loss: -1.4757
Epoch [4/10] Batch [200/313] Loss: -1.5604
Epoch [4/10] Batch [300/313] Loss: -1.1479
Epoch [4/10], Average Loss: -1.3625
Epoch [5/10] Batch [100/313] Loss: -1.4153
Epoch [5/10] Batch [200/313] Loss: -1.7031
Epoch [5/10] Batch [300/313] Loss: -1.8447
Epoch [5/10], Average Loss: -1.5949
Epoch [6/10] Batch [100/313] Loss: -1.4191
Epoch [6/10] Batch [200/313] Loss: -1.7792
Epoch [6/10] Batch [300/313] Loss: -2.2872
Epoch [6/10], Average Loss: -1.8452
Epoch [7/1

('              precision    recall  f1-score   support\n\n        ship       0.70      0.96      0.81      1000\n       truck       0.94      0.58      0.72      1000\n\n    accuracy                           0.77      2000\n   macro avg       0.82      0.77      0.76      2000\nweighted avg       0.82      0.77      0.76      2000\n',
 '              precision    recall  f1-score   support\n\n        ship       0.70      0.97      0.81      5000\n       truck       0.95      0.59      0.73      5000\n\n    accuracy                           0.78     10000\n   macro avg       0.83      0.78      0.77     10000\nweighted avg       0.83      0.78      0.77     10000\n')